In [1]:
# ! pip install pandas==1.5.3
# ! pip install numpy==1.23.1

In [86]:
import pandas as pd 
from pyspark.sql.functions import array_contains, array_intersect, array, udf, size
from pyspark.sql.functions import lit , size, col
from pyspark.sql.types import BooleanType
import re 
from bloomberg.ai.librarian import Librarian, get_config
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lit
import pandas as pd
from bs4 import BeautifulSoup


def check_overlap(nicodes):
    to_exclude = [
        # 'BORGBV',
        # 'BORGFF',
        # 'CMPAUTO',
        # 'BORGDONE',
        # 'BORGSCND',
        # 'BORG2',
        # 'BORG2DON',
        # 'CYBORG',
        'HEADS'
    ]
    
    to_include = [
        'MGMTCHG', # Management Change
        'INI', # Company IPOs
        'SECONDBA', # secondary offerings
        'ERN', # Earnings cannot be fully automated, because we never know what companies will put in there
               # We have extraction to pull non-numerical numbers, it’s not job cuts that we look out for 
        'BORDONE', #: automated stories with human intervention 
        'JOBCUTS',  # This is hard, because companies use different wordings to use 
        'CREDITCG'
    ]
    
    nicode_vals = list(map(lambda x: x.nicode_val, nicodes))
    is_excluded = len(set(nicode_vals) & set(to_exclude)) == 0
    is_included = len(set(nicode_vals) & set(to_include)) > 0
    is_included = True
    return is_excluded & is_included


# def check_security_figis(tickers, to_check_list):
    # security_figi_values = set(map(lambda x: x.security_figi, tickers))
    # return len(security_figi_values & to_check_list) > 0


# def check_figi_udf(to_check_list):
# return udf(lambda c: check_security_figis(c, to_check_list), returnType=BooleanType())


check_overlap_udf = udf(f=check_overlap, returnType=BooleanType())
# check_figi_udf = udf(f=check_security_figis, returnType=BooleanType())

23/07/17 19:11:48 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 381,382,383,384,385 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.


In [2]:
config = get_config()
config.spark.properties["spark.executor.instances"] = 30
config.spark.properties["spark.executor.memory"] = "20G"
config.spark.properties["spark.driver.memory"] = "20G"

librarian = Librarian(cfg=config)
spark = librarian.spark_session

newsarchive_df = (
    librarian
        .datasets
        .newsarchive
        .prod()
        .pvf_newsldap_4()
        .pvf_newsldap_6()
        .begin(2022, 1, 1)
        .end(2023, 1, 30)
    .fetch()
)
newsarchive_df = newsarchive_df.filter(newsarchive_df.bloom_lang == 1)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/layers/com.bloomberg.ds.buildpacks.spark/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/layers/com.bloomberg.ds.buildpacks.hadoop/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


:: loading settings :: file = /layers/com.bloomberg.ds.buildpacks.spark/spark/conf/ivysettings.xml


23/07/17 17:56:56 INFO kns= appId= execId= DelegationTokenUtil: Logging in user from delegation token
23/07/17 17:56:56 INFO kns= appId= execId= DelegationTokenUtil: hadoop token File found
23/07/17 17:56:56 INFO kns= appId= execId= DelegationTokenUtil: Loaded 2 tokens
Setting spark.hadoop.yarn.resourcemanager.principal to job
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/17 17:57:40 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
suid_rid_map = pd.read_csv('all_suid_rid_maps.csv.gz', index_col=0).dropna()

23/07/17 17:58:14 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 24,25 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.


In [7]:
suid_rid_map_df = spark.createDataFrame(suid_rid_map) 

In [8]:
# press_release_na_df = (
#     newsarchive_df
#        .join(suid_rid_map_df, suid_rid_map_df.release_suid==newsarchive_df.suid)
#        .select(['story_suid', 'suid', 'wire', 'headline', 'body', 'timeofarrival', 'bunch_id', 'nicodes'])
# )

In [19]:
newsarchive_df.select('tickers', 'derived_tickers').head()

Row(tickers=None, derived_tickers=None)

In [26]:
full_newsarchive_df = (
    newsarchive_df
       .join(suid_rid_map_df, suid_rid_map_df.story_suid==newsarchive_df.suid)
       .select(['release_suid', 'suid', 
                'wire', 'headline', 'body', 'timeofarrival', 'bunch_id', 'nicodes',
                'tickers', 'derived_tickers'
               ])
)

In [27]:
beat_articles_df = (
    full_newsarchive_df
        .filter(full_newsarchive_df.wire.isin([25]))#, 2345]))
        .filter(check_overlap_udf(full_newsarchive_df.nicodes))
)               

In [ ]:
partition_count = 1
file_format = 'json'
output_dir = 'hdfs://DOB2-GEN/user/aspangher/press-release-newsarchive/beat-article-limited.json'
compression = "gzip"
(beat_articles_df
     .repartition(partition_count)
     .write.mode("overwrite")
     .format(file_format)
     .option("compression", compression)
     .save(output_dir)
)

In [ ]:
# ! pip install bloomberg.ai.remoteio
from bloomberg.ai.remoteio import RemoteIO
from bloomberg.ai.remoteio import BCSConfig

RemoteIO.ls('hdfs://DOB2-GEN/user/aspangher/press-release-newsarchive/beat-article-limited.json')

bcs_cfg = BCSConfig(
    endpoint_url="http://s3.dev.obdc.bcs.bloomberg.com",
    access_key="VN2M29BH4PCAJT9ABZKN",
    secret_key="O9bcBpaprrXr6Q3dorn0XYI4Kp8go6oBDBYFYqeD",
    proxy=None
)

RemoteIO.transfer(
    'hdfs://DOB2-GEN/user/aspangher/press-release-newsarchive/beat-article-limited.json',
    's3://aspangher/press-release-newsarchive/beat-article-limited',
    to_storage_config=bcs_cfg
)

# Get Archive 

In [28]:
beat_articles_pdf = beat_articles_df.toPandas()

23/07/17 18:18:40 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 121,122,123,124,125 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.


In [46]:
from more_itertools import flatten

In [55]:
beat_articles_pdf['derived_tickers'][0][0]

Row(ticker_val='ALL@AU', bloomberg_entity_id='BBG001FHYWL2', classification_score=None, company_id=193196, is_added=False, is_anti=False, is_important=True, is_primary=False, roles_allowed=None, rollup_score=None, score=95, score_flag='', security_figi='BBG000G2SMB6', type='0')

23/07/17 18:44:14 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 246,247,248,249,250 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.


In [59]:
(beat_articles_pdf['derived_tickers']
 .apply(lambda x: [] if x is None else x)
 .apply(lambda x: list(map(lambda y: (y.bloomberg_entity_id, y.security_figi), x )))
 .pipe(lambda s: pd.DataFrame(list(flatten(s.tolist()))))
 .groupby(0)[1]
 .aggregate(set)
 .str.len()
 .value_counts()
)

1    1651
2       3
Name: 1, dtype: int64

23/07/17 18:50:21 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 276,277,278,279,280 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.


In [71]:
security_figis = (
    beat_articles_pdf['derived_tickers']
     .apply(lambda x: [] if x is None else x)
     .apply(lambda x: list(map(lambda y: y.security_figi, x)))
     .pipe(lambda s: pd.Series(list(flatten(s.tolist()))))
     .dropna().drop_duplicates()
     .tolist()
)

23/07/17 18:58:32 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 316,317,318,319,320 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.
23/07/17 18:59:33 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 321,322,323,324,325 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.


In [87]:
def check_security_figis(tickers, to_check_list=security_figis):
    security_figi_values = set(map(lambda x: x.security_figi, tickers))
    return len(security_figi_values & to_check_list) > 0


# def check_figi_udf(to_check_list):
    # return udf(lambda c: check_security_figis(c, to_check_list), returnType=BooleanType())

23/07/17 19:12:50 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 386,387,388,389,390 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.


In [88]:
archive_articles_df = (
    full_newsarchive_df
        .filter(full_newsarchive_df.wire.isin([25, 2345]))
        .filter(check_figi_udf(full_newsarchive_df.derived_tickers))
)              

In [91]:
from pyspark.sql.functions import explode

In [102]:
all_derived_tickers = (
    full_newsarchive_df
        .select('suid', explode(full_newsarchive_df.derived_tickers).alias("ticker"))
        .select('suid', 'ticker.security_figi')
)

In [110]:
beat_derived_tickers = (
    beat_articles_df
        .select('suid', explode(beat_articles_df.derived_tickers).alias("ticker"))
        .select('ticker.security_figi')
        .distinct()
)

In [115]:
desired_suids_for_archive = (
    all_derived_tickers
        .join(beat_derived_tickers, on='security_figi', how='inner')
        .select('suid')
        .distinct()
)

In [116]:
archive_df = (
    full_newsarchive_df
        .join(desired_suids_for_archive, on='suid', how='inner')
)

In [117]:
archive_df.count()

23/07/17 19:24:04 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 441,442,443,444,445 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.


116738

In [ ]:
partition_count = 1
file_format = 'json'
output_dir = 'hdfs://DOB2-GEN/user/aspangher/press-release-newsarchive/news-archive-df.json'
compression = "gzip"
(archive_df
     .repartition(partition_count)
     .write.mode("overwrite")
     .format(file_format)
     .option("compression", compression)
     .save(output_dir)
)

In [119]:
# ! pip install bloomberg.ai.remoteio
from bloomberg.ai.remoteio import RemoteIO
from bloomberg.ai.remoteio import BCSConfig

RemoteIO.ls('hdfs://DOB2-GEN/user/aspangher/press-release-newsarchive/')

log4j:WARN No appenders could be found for logger (org.apache.hadoop.fs.FileSystem).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


['hdfs:///user/aspangher/press-release-newsarchive/2022-2023.json',
 'hdfs:///user/aspangher/press-release-newsarchive/beat-article-limited.json',
 'hdfs:///user/aspangher/press-release-newsarchive/news-archive-df.json']

23/07/17 20:27:25 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 751,752,753,754,755 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.


In [120]:
bcs_cfg = BCSConfig(
    endpoint_url="http://s3.dev.obdc.bcs.bloomberg.com",
    access_key="VN2M29BH4PCAJT9ABZKN",
    secret_key="O9bcBpaprrXr6Q3dorn0XYI4Kp8go6oBDBYFYqeD",
    proxy=None
)

RemoteIO.transfer(
    'hdfs://DOB2-GEN/user/aspangher/press-release-newsarchive/news-archive-df.json',
    's3://aspangher/press-release-newsarchive/news-archive-df.json',
    to_storage_config=bcs_cfg
)

23/07/17 20:28:27 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 756,757,758,759,760 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.
23/07/17 20:29:28 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 761,762,763,764,765 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.
23/07/17 20:30:29 WARN kns=aspangher appId=spark-application-1689616617528 execId=driver ExecutorPodsAllocator: Executors with ids 766,767,768,769,770 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt trie

In [7]:
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

def get_hyperlinks(body):
    import os 
    os.system("pip install beautifulsoup4")
    from bs4 import BeautifulSoup
    output = []
    soup = BeautifulSoup(body)
    a_s = soup.find_all('a')
    a_s = list(filter(lambda x: x.attrs.get('href'), a_s))
    
    output = []
    for a in a_s:
        attrs = a.attrs
        attrs['text'] = a.get_text().strip()
        output.append(attrs)
    
    return output
    
get_hyperlinks_udf = udf(f=get_hyperlinks, returnType=ArrayType(
    StructType([
        StructField('href', StringType()),
        StructField('title', StringType()),
        StructField('itemscope', StringType()),
        StructField('itemprop', StringType()),
        StructField('text', StringType()),
    ])
))

full_beat_df = (
    newsarchive_df
     .filter(newsarchive_df.wire == 25)
     .filter(check_overlap_udf(newsarchive_df.nicodes))
)

hyperlinks_df = (
    newsarchive_df
     .filter(newsarchive_df.wire == 25)
     .filter(check_overlap_udf(newsarchive_df.nicodes))
     .limit(2000)
     .select(get_hyperlinks_udf('body').alias('hyperlinks'))
     .toPandas()
)

23/07/07 17:35:13 WARN kns=aspangher appId=spark-application-1688418353205 execId=driver ExecutorPodsAllocator: Executors with ids 27171,27172,27173,27174,27175 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.
23/07/07 17:38:17 WARN kns=aspangher appId=spark-application-1688418353205 execId=driver ExecutorPodsAllocator: Executors with ids 27186,27187,27188,27189,27190 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a previous allocation attempt tried to create them. The executors may have been deleted but the application missed the deletion event.
23/07/07 17:39:18 WARN kns=aspangher appId=spark-application-1688418353205 execId=driver ExecutorPodsAllocator: Executors with ids 27191,27192,27193,27194,27195 were not detected in the Kubernetes cluster after 60000 ms despite the fact that a pr